# TRANSFORMACIÓN DE DATOS

## IMPORTAR PAQUETES

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from category_encoders import TargetEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MaxAbsScaler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

#Automcompletar rápido
%config IPCompleter.greedy=True

## IMPORTAR LOS DATOS

1.- Sustituir la ruta del proyecto.

In [2]:
ruta_proyecto = 'C:/Users/Jorge/Documents/PythonDataScience/35_Machine_Learning_Framework/PROYECTOS_ML/01_LEADSCORING'

2.- Nombrar los ficheros de datos.

In [3]:
nombre_cat = 'cat_resultado_eda.pickle'
nombre_num = 'num_resultado_eda.pickle'

3.- Cargar los datos.

In [4]:
cat = pd.read_pickle(ruta_proyecto + '/02_Datos/03_Trabajo/' + nombre_cat)
num = pd.read_pickle(ruta_proyecto + '/02_Datos/03_Trabajo/' + nombre_num)

4.- Separar la target.

In [5]:
target = num[['compra']].copy().reset_index(drop=True)

In [7]:
num.columns

Index(['compra', 'visitas_total', 'tiempo_en_site_total',
       'paginas_vistas_visita', 'score_actividad', 'score_perfil'],
      dtype='object')

In [8]:
cat.columns

Index(['origen', 'fuente', 'ult_actividad', 'ambito', 'ocupacion',
       'descarga_lm'],
      dtype='object')

## TRANSFORMACIÓN DE CATEGÓRICAS

### One Hot Encoding

#### Variables a aplicar OHE

In [9]:
var_ohe = ['origen','fuente','ult_actividad','ambito', 'ocupacion','descarga_lm']

#### Instanciar

In [10]:
ohe = OneHotEncoder(sparse = False, handle_unknown='ignore')

#### Entrenar y aplicar

In [11]:
cat_ohe = ohe.fit_transform(cat[var_ohe])

#### Guardar como dataframe

In [12]:
cat_ohe = pd.DataFrame(cat_ohe, columns = ohe.get_feature_names_out())

## TRANSFORMACIÓN DE NUMÉRICAS

## UNIFICAR DATASETS TRANSFORMADOS

In [15]:
df = pd.concat([cat_ohe, num.reset_index()], axis =1)

## REESCALAR VARIABLES

### Con Min-Max

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4865 entries, 0 to 4864
Data columns (total 44 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   origen_API                                4865 non-null   float64
 1   origen_Landing Page Submission            4865 non-null   float64
 2   origen_Lead Add Form                      4865 non-null   float64
 3   origen_OTROS                              4865 non-null   float64
 4   fuente_Chat                               4865 non-null   float64
 5   fuente_Direct Traffic                     4865 non-null   float64
 6   fuente_Google                             4865 non-null   float64
 7   fuente_OTROS                              4865 non-null   float64
 8   fuente_Organic Search                     4865 non-null   float64
 9   fuente_Reference                          4865 non-null   float64
 10  ult_actividad_Chat Conversation     

#### Variables a reescalar con Min-Max

In [42]:
var_mms = df.iloc[:,39:].columns

#### Instanciar

In [43]:
mms = MinMaxScaler()

#### Entrenar y aplicar

In [44]:
df_mms = mms.fit_transform(df[var_mms])

#### Guardar como dataframe

In [45]:
#Añadir sufijos a los nombres
nombres_mms = [variable + '_mms' for variable in var_mms]

#Guardar como dataframe
df_mms = pd.DataFrame(df_mms,columns = nombres_mms)

## UNIFICAR DATASETS REESCALADOS

In [55]:
id = df.id #Tenemos que crear poner el id porque no aparece en ninguno de los otros datasets

In [56]:
incluir = [id,cat_ohe, df_mms, target]

### Unir todos los dataframes en el tablón analítico

In [58]:
df_tablon = pd.concat(incluir, axis = 1).set_index('id')

In [60]:
#Comprobaciones
#df_tablon.isna().sum()

origen_API                                  0
origen_Landing Page Submission              0
origen_Lead Add Form                        0
origen_OTROS                                0
fuente_Chat                                 0
fuente_Direct Traffic                       0
fuente_Google                               0
fuente_OTROS                                0
fuente_Organic Search                       0
fuente_Reference                            0
ult_actividad_Chat Conversation             0
ult_actividad_Converted to Lead             0
ult_actividad_Email Link Clicked            0
ult_actividad_Email Opened                  0
ult_actividad_OTROS                         0
ult_actividad_Page Visited on Website       0
ult_actividad_SMS Sent                      0
ambito_Banking, Investment And Insurance    0
ambito_Business Administration              0
ambito_Finance Management                   0
ambito_Healthcare Management                0
ambito_Human Resource Management  

In [59]:
#df_tablon.describe().T

,count,mean,std,min,25%,50%,75%,max
origen_API,4865.0,0.308325,0.461849,0.0,0.000000,0.000000,1.000000,1.0
origen_Landing Page Submission,4865.0,0.635355,0.481380,0.0,0.000000,1.000000,1.000000,1.0
origen_Lead Add Form,4865.0,0.049332,0.216582,0.0,0.000000,0.000000,0.000000,1.0
origen_OTROS,4865.0,0.006989,0.083314,0.0,0.000000,0.000000,0.000000,1.0
fuente_Chat,4865.0,0.061048,0.239444,0.0,0.000000,0.000000,0.000000,1.0
fuente_Direct Traffic,4865.0,0.324974,0.468413,0.0,0.000000,0.000000,1.000000,1.0
fuente_Google,4865.0,0.392395,0.488334,0.0,0.000000,0.000000,1.000000,1.0
fuente_OTROS,4865.0,0.028777,0.167196,0.0,0.000000,0.000000,0.000000,1.0
fuente_Organic Search,4865.0,0.147379,0.354520,0.0,0.000000,0.000000,0.000000,1.0
fuente_Reference,4865.0,0.045427,0.208259,0.0,0.000000,0.000000,0.000000,1.0


## GUARDAR DATASET TRAS TRANSFORMACIÓN DE DATOS

En formato pickle para no perder las modificaciones de metadatos.

In [63]:
#Definir los nombres del archivo
ruta_df_tablon = ruta_proyecto + '/02_Datos/03_Trabajo/' + 'df_tablon.pickle'

In [64]:
#Guardar los archivos
df_tablon.to_pickle(ruta_df_tablon)